# Praktikum 1.2 Natural Language Processing 

### Nama : Ronggur Mahendra Widya Putra
### NIM : 13519008

In [1]:
#Import Libraries
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from keras.layers import Dense
# from keras.utils.vis_utils import plot_model
from tensorflow.keras.utils import plot_model

import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow import keras
from sklearn.metrics import accuracy_score

In [2]:
# Load Train data
train_df = pd.read_parquet('./data/train-00000-of-00001-04b49ae22f595095.parquet', engine='pyarrow')
train_df.head(10)

,text,label
0,− Scope 3: Optional scope that includes indire...,1
1,The Group is not aware of any noise pollution ...,0
2,Global climate change could exacerbate certain...,0
3,Setting an investment horizon is part and parc...,0
4,Climate change the physical impacts of climate...,0
5,Projects with potential limited adverse social...,0
6,We emitted 13.4 million tonnes CO2 of Scope 2 ...,1
7,We do not provide normalised figures for our C...,1
8,We anticipate that the potential effects of cl...,0
9,Enhancing our responsible screening criteria N...,0


In [3]:
train_df.describe()

,label
count,1000.000000
mean,0.908000
std,0.764278
min,0.000000
25%,0.000000
50%,1.000000
75%,1.250000
max,2.000000


In [4]:
train_df_data  = train_df['text'].to_list()
train_df_label = train_df['label'].to_list()

In [5]:
# Split data
train_data, val_data, train_label, val_label  = train_test_split(train_df_data, train_df_label, test_size=0.2, random_state=230907)


In [6]:
# load test data
test_df = pd.read_parquet('./data/test-00000-of-00001-3f9f7af4f5914b8e.parquet', engine='pyarrow')
test_df.head(10)


,text,label
0,Sustainable strategy ‘red lines’ For our susta...,0
1,"Verizon’s environmental, health and safety man...",1
2,"In 2019, the Company closed a series of transa...",1
3,"In December 2020, the AUC approved the Electri...",0
4,"Finally, there is a reputational risk linked t...",0
5,Ecoefficiency Eco-efficiency management provid...,1
6,The Group and its customers are exposed to cli...,0
7,Both our Board and executive leadership team r...,1
8,Although it is intended that governments will ...,1
9,Climate-related risks and opportunities have g...,0


In [7]:
test_data  = test_df['text'].to_list()
test_label = test_df['label'].to_list()

In [8]:
print("train_label : ", len(train_data))
print("train_label : ",len(train_label))

print("val_label : ", len(val_data))
print("val_label : ",len(val_label))

print("test_data : ",len(test_data))
print("test_label :",len(test_label))


train_label :  800
train_label :  800
val_label :  200
val_label :  200
test_data :  320
test_label : 320


In [9]:
# Preprocess & Tokenize
MAX_WORDS = 10000
tokenizer = Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(texts = train_data)

train_sequences = tokenizer.texts_to_sequences(train_data)
val_sequences = tokenizer.texts_to_sequences(val_data)
test_sequences = tokenizer.texts_to_sequences(test_data)

train_label = np.array(train_label)
val_label = np.array(val_label)
test_label = np.array(test_label)

In [10]:
# Tokenize
train_data_tokenized = pad_sequences(train_sequences, maxlen = 100)
val_data_tokenized = pad_sequences(val_sequences, maxlen = 100)
test_data_tokenized = pad_sequences(test_sequences, maxlen = 100)

# Cast into numpy array 
train_data_tokenized = np.array(train_data_tokenized)
val_data_tokenized = np.array(val_data_tokenized)
test_data_tokenized = np.array(test_data_tokenized)

# RNN/LSTM MODEL

In [11]:
# Define Model
# Hyper parameter sama dengan contoh di slide
model_rnn = Sequential()
model_rnn.add(Embedding(input_dim = MAX_WORDS, output_dim = 128, input_length = train_data_tokenized.shape[1]))
model_rnn.add(Bidirectional(LSTM(64, return_sequences=True)))
model_rnn.add(Bidirectional(LSTM(32)))
model_rnn.add(Dense(1,activation='sigmoid'))


In [12]:
#compile model
model_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_rnn.summary())
print("\n\nModel Visualize")
plot_model(model_rnn, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          1280000   
                                                                 
 bidirectional (Bidirection  (None, 100, 128)          98816     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 1420097 (5.42 MB)
Trainable params: 1420097 (5.42 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


Model Visua

In [13]:
# Train
model_rnn.fit(train_data_tokenized, train_label, epochs=10, batch_size=32, validation_data=(val_data_tokenized, val_label))

Epoch 1/10
25/25 [==============================] - 24s 360ms/step - loss: 0.2959 - accuracy: 0.4087 - val_loss: 0.5728 - val_accuracy: 0.4350
Epoch 2/10
25/25 [==============================] - 5s 189ms/step - loss: -0.4261 - accuracy: 0.4250 - val_loss: -0.5450 - val_accuracy: 0.5000
Epoch 3/10
25/25 [==============================] - 5s 191ms/step - loss: -1.9914 - accuracy: 0.5775 - val_loss: -1.4968 - val_accuracy: 0.6500
Epoch 4/10
25/25 [==============================] - 5s 199ms/step - loss: -3.2675 - accuracy: 0.6700 - val_loss: -2.0075 - val_accuracy: 0.6600
Epoch 5/10
25/25 [==============================] - 5s 190ms/step - loss: -3.9906 - accuracy: 0.6800 - val_loss: -2.0686 - val_accuracy: 0.6700
Epoch 6/10
25/25 [==============================] - 5s 193ms/step - loss: -4.6382 - accuracy: 0.7100 - val_loss: -2.1485 - val_accuracy: 0.6600
Epoch 7/10
25/25 [==============================] - 5s 193ms/step - loss: -5.2014 - accuracy: 0.7125 - val_loss: -2.3578 - val_accuracy: 

In [14]:
# Evaluate

loss, acc = model_rnn.evaluate(test_data_tokenized, test_label)
print("loss: ", loss)
print("accuracy: ", acc)

10/10 [==============================] - 1s 62ms/step - loss: -1.6383 - accuracy: 0.6469
loss:  -1.6383483409881592
accuracy:  0.6468750238418579


In [15]:
# Prediction
prediction = model_rnn.predict(test_data_tokenized[:5])

for text, prediction, groundtruth in zip(tokenizer.sequences_to_texts(test_data_tokenized), prediction, test_label[:5]):
    sentiment = "positive" if prediction > 0.5 else "negative"
    groundtruth = "positive" if groundtruth == 0.5 else "negative"
    print(f"Text: {text} \n Predicted Sentiment: {sentiment}\n Groundtruth: {groundtruth}\n\n")

1/1 [==============================] - 4s 4s/step
Text: sustainable strategy for our sustainable strategy range we incorporate a series of proprietary in order to ensure the performing companies from an esg perspective are not eligible for investment 
 Predicted Sentiment: positive
 Groundtruth: negative


Text: environmental health and safety management system provides a framework for identifying and reducing the risks associated with the environments in which we operate regular management system assessments internal and third party compliance and inspections are performed annually at of facilities worldwide the goal of these assessments is to identify and correct site specific issues and to educate and empower facility managers and supervisors to implement corrective actions environment health and safety efforts are and supported by experienced experts around the world that support our operations and facilities 
 Predicted Sentiment: positive
 Groundtruth: negative


Text: in 2019 th

# Word2Vec Embedding

In [16]:
from gensim.models import Word2Vec

In [17]:
word2vec_model = Word2Vec(sentences=train_data, vector_size=128, window = 5, min_count=1, sg=0)

word2vec_model.save("word2vec.model")

In [18]:
embedding_matrix = np.zeros((MAX_WORDS, 128))
for word,i in tokenizer.word_index.items():
    if i < MAX_WORDS:
        if word in word2vec_model.wv:
            embedding_matrix[i] = word2vec_model.wv[word]

In [19]:
# define model
# Hyper parameter sama dengan contoh di slide
word2vec_model = Sequential()
word2vec_model.add(Embedding(input_dim = MAX_WORDS, output_dim = 128, input_length = train_data_tokenized.shape[1], weights= [embedding_matrix], trainable = True ))
word2vec_model.add(Bidirectional(LSTM(64, return_sequences=True)))
word2vec_model.add(Bidirectional(LSTM(32)))
word2vec_model.add(Dense(1,activation='sigmoid'))

In [20]:
#compile model
word2vec_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(word2vec_model.summary())
print("\n\nModel Visualize")
plot_model(word2vec_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 128)          1280000   
                                                                 
 bidirectional_2 (Bidirecti  (None, 100, 128)          98816     
 onal)                                                           
                                                                 
 bidirectional_3 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1420097 (5.42 MB)
Trainable params: 1420097 (5.42 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


Model Vis

In [21]:
# Train
word2vec_model.fit(train_data_tokenized, train_label, epochs=10, batch_size=32, validation_data=(val_data_tokenized, val_label))

Epoch 1/10
25/25 [==============================] - 23s 350ms/step - loss: 0.3083 - accuracy: 0.4038 - val_loss: 0.3667 - val_accuracy: 0.4350
Epoch 2/10
25/25 [==============================] - 5s 184ms/step - loss: -0.7952 - accuracy: 0.4363 - val_loss: -0.9207 - val_accuracy: 0.6550
Epoch 3/10
25/25 [==============================] - 5s 187ms/step - loss: -1.9854 - accuracy: 0.6150 - val_loss: -1.0480 - val_accuracy: 0.5700
Epoch 4/10
25/25 [==============================] - 5s 187ms/step - loss: -2.8145 - accuracy: 0.6475 - val_loss: -1.5168 - val_accuracy: 0.6800
Epoch 5/10
25/25 [==============================] - 5s 184ms/step - loss: -3.2401 - accuracy: 0.6700 - val_loss: -2.2172 - val_accuracy: 0.6550
Epoch 6/10
25/25 [==============================] - 4s 172ms/step - loss: -4.3133 - accuracy: 0.6988 - val_loss: -2.6830 - val_accuracy: 0.6750
Epoch 7/10
25/25 [==============================] - 4s 173ms/step - loss: -4.7837 - accuracy: 0.6988 - val_loss: -1.3259 - val_accuracy: 

In [22]:
# Evaluate

loss, acc = word2vec_model.evaluate(test_data_tokenized, test_label)
print("loss: ", loss)
print("accuracy: ", acc)

10/10 [==============================] - 1s 57ms/step - loss: -1.9364 - accuracy: 0.6344
loss:  -1.9364475011825562
accuracy:  0.6343749761581421


In [23]:
# Prediction
prediction = word2vec_model.predict(test_data_tokenized[:5])

for text, prediction, groundtruth in zip(tokenizer.sequences_to_texts(test_data_tokenized), prediction, test_label[:5]):
    sentiment = "positive" if prediction > 0.5 else "negative"
    groundtruth = "positive" if groundtruth == 0.5 else "negative"
    print(f"Text: {text} \n Predicted Sentiment: {sentiment}\n Groundtruth: {groundtruth}\n\n")

1/1 [==============================] - 4s 4s/step
Text: sustainable strategy for our sustainable strategy range we incorporate a series of proprietary in order to ensure the performing companies from an esg perspective are not eligible for investment 
 Predicted Sentiment: positive
 Groundtruth: negative


Text: environmental health and safety management system provides a framework for identifying and reducing the risks associated with the environments in which we operate regular management system assessments internal and third party compliance and inspections are performed annually at of facilities worldwide the goal of these assessments is to identify and correct site specific issues and to educate and empower facility managers and supervisors to implement corrective actions environment health and safety efforts are and supported by experienced experts around the world that support our operations and facilities 
 Predicted Sentiment: negative
 Groundtruth: negative


Text: in 2019 th

# Attention Based Model

In [24]:
from transformers import DistilBertTokenizer, TFDistilBertModel, TFAutoModel, AutoTokenizer
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling1D, Attention, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [25]:
# Download pretrained language model DistilBERT
model_name = "distilbert-base-uncased"
attention_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# attention_encoder = TFAutoModel.from_pretrained(model_name)
attention_model = TFDistilBertModel.from_pretrained(model_name)


In [26]:
attention_model.summary()

Model: "tf_distil_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
Total params: 66362880 (253.15 MB)
Trainable params: 66362880 (253.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
from transformers import DistilBertTokenizer, TFDistilBertModel
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = TFDistilBertModel.from_pretrained("distilbert-base-uncased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer.tokenize(text)
output = model(encoded_input)


AssertionError: Exception encountered when calling layer 'distilbert' (type TFDistilBertMainLayer).

Too many inputs.

Call arguments received by layer 'distilbert' (type TFDistilBertMainLayer):
  • inputs=["'replace'", "'me'", "'by'", "'any'", "'text'", "'you'", '"\'"', "'d'", "'like'", "'.'"]
  • attention_mask=None
  • head_mask=None
  • training=False

In [30]:
# Convert NumPy arrays to lists
# train_sequences = train_sequences.tolist()
# val_sequences = val_sequences.tolist()
# test_sequences = test_sequences.tolist()
# train_label = train_label.tolist()
# val_label = val_label.tolist()
# test_label = test_label.tolist()

text = "Replace me by any text you'd like."
encoded_input = attention_tokenizer.tokenize(text)



In [34]:
# # # train/finetune the model
# train_data_tokenized = pad_sequences(train_sequences, maxlen = 100)
# val_data_tokenized = pad_sequences(val_sequences, maxlen = 100)
# test_data_tokenized = pad_sequences(test_sequences, maxlen = 100)
train_data_tokenized = attention_tokenizer.tokenize(train_data_tokenized)
val_data_tokenized = attention_tokenizer.tokenize(val_data_tokenized)
test_data_tokenized = attention_tokenizer.tokenize(test_data_tokenized)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
attention_model.fit(train_data_tokenized, train_label, validation_data=(val_data_tokenized, val_label), epochs=3, batch_size=32)

In [35]:
attention_model.predict(encoded_input)

ValueError: in user code:

    File "C:\Users\rongg\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\rongg\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\rongg\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\rongg\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2283, in predict_step
        return self(x, training=False)
    File "C:\Users\rongg\anaconda3\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\rongg\AppData\Local\Temp\__autograph_generated_file5f171465.py", line 11, in tf__call
        outputs = ag__.converted_call(ag__.

# Report

## Performance

- LSTM MODEL

    Training Accucacy : 0.71

    Test Accucacy : 0.70

- Word2Vec Embedding

    Training Accucacy : 0.70

    Test Accucacy : 0.64
    

https://huggingface.co/distilbert-base-uncased